In [1]:
import itk
import vtk
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
PixelType = itk.F

In [2]:
fixed = itk.imread("Data/case6_gre1.nrrd",PixelType)

In [3]:
moving= itk.imread("Data/case6_gre2.nrrd",PixelType)

In [4]:
dimension = fixed.GetImageDimension()
FixedImageType = type(fixed)
MovingImageType = type(moving)
MovingImageType

itk.itkImagePython.itkImageF3

## Recalage avec translationTransform ##

In [230]:
#TransformType = itk.Rigid3DTransform[itk.D]
TransformType = itk.TranslationTransform[itk.D,dimension]
initial_transform = TransformType.New()

In [231]:
optimizer = itk.RegularStepGradientDescentOptimizerv4.New()
optimizer.SetLearningRate(4.0)
optimizer.SetMinimumStepLength(0.001)
optimizer.SetNumberOfIterations(20)

In [232]:
metric = itk.MeanSquaresImageToImageMetricv4[FixedImageType, MovingImageType].New()

In [233]:
fixed_interpolation = itk.LinearInterpolateImageFunction[FixedImageType, itk.D].New()
metric.SetFixedInterpolator(fixed_interpolation)

In [234]:
registration = itk.ImageRegistrationMethodv4[FixedImageType, MovingImageType].New()
registration.SetMetric(metric)
registration.SetOptimizer(optimizer)
registration.SetFixedImage(fixed)
registration.SetMovingImage(moving)
registration.SetInitialTransform(initial_transform)

In [235]:
moving_initial_transform = TransformType.New()
initial_parameters = moving_initial_transform.GetParameters()
initial_parameters[0] = 0
initial_parameters[1] = 0
initial_parameters[2] = 0
moving_initial_transform.SetParameters(initial_parameters)
registration.SetMovingInitialTransform(moving_initial_transform)

In [236]:
identity_transform = TransformType.New()
identity_transform.SetIdentity()
registration.SetFixedInitialTransform(identity_transform)
registration.SetNumberOfLevels(1)

In [237]:
registration.Update()

In [238]:
transform = registration.GetTransform()
final_parameters = transform.GetParameters()
print("Translation X: " +str(final_parameters.GetElement(0)))
print("Translation Y: " +str(final_parameters.GetElement(1)))
print("Translation Z: " +str(final_parameters.GetElement(2)))
optimizer.GetValue()

Translation X: -0.6951590472781529
Translation Y: -3.4733086775013957
Translation Z: -59.539168082382226


5906.019774094595

In [239]:
CompositeTransformType = itk.CompositeTransform[itk.D, dimension]
output_composite_transform = CompositeTransformType.New()
output_composite_transform.AddTransform(moving_initial_transform)
output_composite_transform.AddTransform(registration.GetModifiableTransform())

resampler = itk.ResampleImageFilter.New(Input=moving, Transform=transform, UseReferenceImage=True,
                                            ReferenceImage=fixed)
resampler.SetDefaultPixelValue(100)
resampler.Update()
resampling1 = resampler.GetOutput()

## Recalage avec BSplineTransform ##

In [119]:
TransformType = itk.BSplineTransform[itk.D,dimension,dimension]
initial_transform = TransformType.New()

In [120]:
optimizer = itk.LBFGSOptimizerv4.New()
optimizer.SetGradientConvergenceTolerance(0.05);
optimizer.SetLineSearchAccuracy(0.9);
optimizer.SetDefaultStepLength(.5);
optimizer.TraceOn();
optimizer.SetMaximumNumberOfFunctionEvaluations(1000);

In [121]:
scales = itk.OptimizerParameters[itk.D](initial_transform.GetNumberOfParameters())
scales.Fill(1.0)
optimizer.SetScales(scales)

In [122]:
metric = itk.MeanSquaresImageToImageMetricv4[FixedImageType, MovingImageType].New()

In [123]:
fixed_interpolation = itk.LinearInterpolateImageFunction[FixedImageType, itk.D].New()
metric.SetFixedInterpolator(fixed_interpolation)

In [124]:
registration = itk.ImageRegistrationMethodv4[FixedImageType, MovingImageType].New()
registration.SetMetric(metric)
registration.SetOptimizer(optimizer)
registration.SetFixedImage(fixed)
registration.SetMovingImage(moving)
registration.SetInitialTransform(initial_transform)

In [125]:
registration.Update()

In [126]:
transform = registration.GetTransform()
coefficients = transform.GetParameters()
print("B-Spline coefficients: ", coefficients)
print("Number of coefficients: ", len(coefficients))
optimizer.GetValue()

B-Spline coefficients:  <itk.itkOptimizerParametersPython.itkOptimizerParametersD; proxy of <Swig Object of type 'itkOptimizerParametersD *' at 0x00000215041C4C00> >
Number of coefficients:  192


55130.199734167254

In [42]:
resampler = itk.ResampleImageFilter.New(Input=moving, Transform=transform, UseReferenceImage=True,
                                            ReferenceImage=fixed)
resampler.SetDefaultPixelValue(100)
resampler.Update()
resampling = resampler.GetOutput()

In [349]:
subtraction2 = itk.SubtractImageFilter(Input1=fixed, Input2=resampler)
type(subtraction2)

itk.itkImagePython.itkImageF3

## Recalage Euler3DTransform ##

In [324]:
TransformType = itk.Euler3DTransform[itk.D]
initial_transform = TransformType.New()

In [325]:
optimizer = itk.RegularStepGradientDescentOptimizerv4.New()
optimizer.SetLearningRate(4.0)
optimizer.SetMinimumStepLength(0.001)
optimizer.SetNumberOfIterations(20)

In [326]:
metric = itk.MeanSquaresImageToImageMetricv4[FixedImageType, MovingImageType].New()

In [327]:
fixed_interpolation = itk.LinearInterpolateImageFunction[FixedImageType, itk.D].New()
metric.SetFixedInterpolator(fixed_interpolation)

In [328]:
registration = itk.ImageRegistrationMethodv4[FixedImageType, MovingImageType].New()
registration.SetMetric(metric)
registration.SetOptimizer(optimizer)
registration.SetFixedImage(fixed)
registration.SetMovingImage(moving)
registration.SetInitialTransform(initial_transform)

In [329]:
moving_initial_transform = TransformType.New()
initial_parameters = moving_initial_transform.GetParameters()
initial_parameters[3] = 0
initial_parameters[4] = 0
initial_parameters[5] = 0
moving_initial_transform.SetParameters(initial_parameters)
registration.SetMovingInitialTransform(moving_initial_transform)

In [330]:
fixed_parameters = moving_initial_transform.GetFixedParameters()
fixed_parameters[0] = moving.GetLargestPossibleRegion().GetSize()[0] / 2.0
fixed_parameters[1] = moving.GetLargestPossibleRegion().GetSize()[1] / 2.0
fixed_parameters[2] = moving.GetLargestPossibleRegion().GetSize()[2] / 2.0
moving_initial_transform.SetFixedParameters(fixed_parameters)

In [331]:
identity_transform = TransformType.New()
identity_transform.SetIdentity()
registration.SetFixedInitialTransform(identity_transform)
registration.SetNumberOfLevels(1)

In [332]:
registration.Update()

In [333]:
transform = registration.GetTransform()
final_parameters = transform.GetParameters()
optimizer.GetValue()

1.7976931348623157e+308

In [334]:
CompositeTransformType = itk.CompositeTransform[itk.D, dimension]
output_composite_transform = CompositeTransformType.New()
output_composite_transform.AddTransform(moving_initial_transform)
output_composite_transform.AddTransform(registration.GetModifiableTransform())

resampler = itk.ResampleImageFilter.New(Input=moving, Transform=transform, UseReferenceImage=True,
                                            ReferenceImage=fixed)
resampler.SetDefaultPixelValue(100)
resampler.Update()
resampling2 = resampler.GetOutput()

In [294]:
subtraction3 = itk.SubtractImageFilter(Input1=fixed, Input2=resampler)
type(subtraction3)

itk.itkImagePython.itkImageF3

## Partie Segmentation

In [ ]:
import sys
import itk
import os
import matplotlib
import matplotlib.pyplot as plt

def segment_tumor(input_filepath, output_filepath, seed, lower, upper, image_nb=50):
    # Read the input image
    input_image = itk.imread(input_filepath, pixel_type=itk.F)

    # Apply anisotropic diffusion filter for smoothing
    smoother = itk.GradientAnisotropicDiffusionImageFilter.New(
        Input=input_image, NumberOfIterations=20, TimeStep=0.04, ConductanceParameter=3)
    smoother.Update()

    # Display the smoothed image to choose a seed point
    plt.ion()
    plt.imshow(itk.GetArrayViewFromImage(smoother.GetOutput())[:, image_nb, :], cmap="gray")
    plt.title("Select seed point and press a key")
    plt.waitforbuttonpress()

    # Instantiate the ConnectedThresholdImageFilter
    connected_threshold = itk.ConnectedThresholdImageFilter.New(smoother.GetOutput())
    connected_threshold.SetReplaceValue(255)
    connected_threshold.SetLower(lower)
    connected_threshold.SetUpper(upper)

    # Set seed point for segmentation
    new_seed = (seed[0], image_nb, seed[2])
    connected_threshold.SetSeed(new_seed)
    connected_threshold.Update()
    # Rescale the intensity of the segmented output
    in_type = itk.output(connected_threshold)
    dimension = input_image.GetImageDimension()
    output_type = itk.Image[itk.UC, dimension]
    rescaler = itk.RescaleIntensityImageFilter[in_type, output_type].New(connected_threshold)
    itk.imwrite(rescaler, output_filepath)

    # Display the segmented output
    plt.imshow(itk.GetArrayViewFromImage(connected_threshold.GetOutput())[:, image_nb, :], cmap="gray")
    plt.title("Segmented tumor")
    #plt.waitforbuttonpress()

In [ ]:
matplotlib.use('TkAgg')

# File paths
scan1_filepath = "Data/case6_gre1.nrrd"
scan2_filepath = "Data/case6_gre2.nrrd"
output1_filepath = 'case6_gre1_segmented.nrrd'
output2_filepath = 'case6_gre2_segmented.nrrd'

# Segmentation parameters (example values)
seed1 = (120, 105, 10)  # Adjust these coordinates for the first scan
seed2 = (120, 105, 30)  # Adjust these coordinates for the second scan
lower, upper = 20, 255

# Segment tumors in both scans
segment_tumor(scan1_filepath, output1_filepath, seed1, lower, upper)
#segment_tumor(scan2_filepath, output2_filepath, seed2, lower, upper)

## Affichage de la tumeur ##

In [335]:
source = itk.vtk_image_from_image(resampling2)
source2 = itk.vtk_image_from_image(resampling1)

In [336]:
renderWindow = vtk.vtkRenderWindow()
renderer = vtk.vtkRenderer()

In [337]:
renderWindow.AddRenderer(renderer)
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetInteractorStyle(vtk.vtkInteractorStyleTrackballCamera())
renderWindow.SetInteractor(renderWindowInteractor)

In [338]:
# Create a vtk.vtkContourFilter to do the isocontouring
# Set the input to the output of the reader
# Set the value to 135
contour = vtk.vtkContourFilter()
contour.SetInputData(source)
contour.SetValue(0, 135)

# This is the vtk.vtkPolyDataMapper
contourMapper = vtk.vtkPolyDataMapper()

# Connect the mapper to the contour filter
# Remember to turn ScalarVisibilityOff()
contourMapper.SetInputConnection(contour.GetOutputPort())
contourMapper.ScalarVisibilityOff()

# This is the vtk.vtkActor
contourActor = vtk.vtkActor()

# Set the mapper
contourActor.SetMapper(contourMapper)

# Add the actor
renderer.AddActor(contourActor)

In [262]:
# Create a vtk.vtkContourFilter to do the isocontouring
# Set the input to the output of the reader
# Set the value to 135
contour2 = vtk.vtkContourFilter()
contour2.SetInputData(source2)
contour2.SetValue(0, 135)

# This is the vtk.vtkPolyDataMapper
contourMapper2 = vtk.vtkPolyDataMapper()

# Connect the mapper to the contour filter
# Remember to turn ScalarVisibilityOff()
contourMapper2.SetInputConnection(contour2.GetOutputPort())
contourMapper2.ScalarVisibilityOff()

# This is the vtk.vtkActor
contourActor2 = vtk.vtkActor()

# Set the mapper
contourActor2.SetMapper(contourMapper2)

# Add the actor
renderer.AddActor(contourActor2)

In [339]:
# Create an opacity transfer function to map
# scalar value to opacity
opacityFun = vtk.vtkPiecewiseFunction()

# Set a mapping going from 0.0 opacity at 90, up to 0.2 at 100,
# and back down to 0.0 at 120.
opacityFun.AddPoint(0.0, 0.0)
opacityFun.AddPoint(90.0, 0.0)
opacityFun.AddPoint(100.0, 0.2)
opacityFun.AddPoint(120.0, 0.0)

# Create a color transfer function for the mapping of scalar
# value into color
colorFun = vtk.vtkColorTransferFunction()

# Set the color to a constant value, you might
# want to try (0.8, 0.4, 0.2)
colorFun.AddRGBPoint(0.0, .8, .4, .2)
colorFun.AddRGBPoint(255.0, .8, .4, .2)

# Create a volume property
# Set the opacity and color. Change interpolation
# to linear for a more pleasing image
property = vtk.vtkVolumeProperty()
property.SetScalarOpacity(opacityFun)
property.SetColor(colorFun)
property.SetInterpolationTypeToLinear()

In [340]:
# Create the volume mapper
mapper = vtk.vtkSmartVolumeMapper()

# Set the input to the output of the reader
mapper.SetInputData(source)

# Create the volume
volume = vtk.vtkVolume()

# Set the property and the mapper
#volume.SetProperty(property)
volume.SetMapper(mapper)

# Add the volume to the renderer
renderer.AddVolume(volume)

# Render and start the interactor
renderWindow.Render()
renderWindowInteractor.Start()